In [1]:
# GPU 런타임 설정을 한 후 실행한다.
!pip install konlpy


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 패키지 설치
import konlpy
import pandas as pd
import numpy as np
# from konlpy.tag import Twitter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn import model_selection, metrics

In [3]:
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   152  100   152    0     0    161      0 --:--:-- --:--:-- --:--:--   161

100    69    0    69    0     0     46      0 --:--:--  0:00:01 --:--:--    46

100   297  100   297    0     0    159      0  0:00:01  0:00:01 --:--:--   159
100   297  100   297    0     0    159      0  0:00:01  0:00:01 --:--:--     0

  0 14.0M    0  112k    0     0  43740      0  0:05:37  0:00:02  0:05:35 43740
 62 14.0M   62 9088k    0     0  2509k      0  0:00:05  0:00:03  0:00:02 8976k
100 14.0M  100 14.0M    0     0  3446k      0  0:00:04  0:00:04 --:--:-- 9150k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload 

### keep_default_na=False
- https://velog.io/@hamdoe/pd.readcsv-%EC%8B%9C-NaN-%EA%B0%92-None%EC%9C%BC%EB%A1%9C-%EB%B0%94%EA%BE%B8%EA%B8%B0
- https://atotw.tistory.com/485

In [4]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)


In [5]:
df_train[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [7]:
# text_train, y_train = df_train['document'].as_matrix(), df_train['label'].as_matrix()
# text_test, y_test = df_test['document'].as_matrix(), df_test['label'].as_matrix()
text_train, y_train = df_train['document'].to_numpy(), df_train['label'].to_numpy()
text_test, y_test = df_test['document'].to_numpy(), df_test['label'].to_numpy()


# twitter_tag = Twitter()
twitter_tag = Okt()

In [8]:
# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

## 문서-단어 행렬 
- 시간 많이 걸림(13분 정도)

In [9]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer, 
                     max_features=3000)
X_train = cv.fit_transform(text_train)
X_test = cv.transform(text_test)

In [10]:
X_train.shape, y_train.shape

((150000, 3000), (150000,))

In [11]:
X_test.shape, y_test.shape

((50000, 3000), (50000,))

In [12]:
X_train[:1]

<1x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [13]:
X_train[:1].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [14]:
X_train[:1].A

array([[0., 0., 0., ..., 0., 0., 0.]])

In [15]:
X_train.shape, X_train.A.shape

((150000, 3000), (150000, 3000))

In [16]:
y_test[:3]

array([1, 0, 0], dtype=int64)

In [18]:
# feature_names = cv.get_feature_names()

feature_names = cv.get_feature_names()

In [19]:
type(feature_names)

list

In [20]:
len(feature_names)

3000

In [21]:
feature_names

['!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!',
 '"',
 '""',
 '"""',
 '&',
 "'",
 '(',
 '(?)',
 ')',
 '+',
 ',',
 ',,',
 ',,,',
 ',,,,',
 ',.',
 '-',
 '--',
 '-_-',
 '.',
 '."',
 '.,',
 '..',
 '...',
 '....',
 '.....',
 '......',
 '.......',
 '........',
 '.........',
 '..........',
 '...?',
 '..;;',
 '..?',
 '..^^',
 '.^^',
 '/',
 '//',
 '0',
 '007',
 '1',
 '10',
 '100',
 '100%',
 '10년',
 '10분',
 '11',
 '12',
 '13',
 '15',
 '18',
 '19',
 '1시간',
 '2',
 '20',
 '2014년',
 '20년',
 '20분',
 '21',
 '2시간',
 '3',
 '30',
 '30년',
 '30분',
 '4',
 '40',
 '5',
 '5분',
 '6',
 '7',
 '70년',
 '8',
 '80년',
 '9',
 '90년',
 ':',
 ':)',
 ';',
 ';;',
 ';;;',
 ';;;;',
 '<',
 '=',
 '>',
 '?',
 '?!',
 '?...',
 '??',
 '???',
 '????',
 '@',
 '[',
 ']',
 '^',
 '^^',
 '_',
 '`',
 'a',
 'b',
 'bad',
 'c',
 'cg',
 'cgv',
 'd',
 'dvd',
 'ebs',
 'good',
 'i',
 'kbs',
 'm',
 'mbc',
 'ocn',
 'of',
 'oo',
 'ooo',
 'ost',
 's',
 'sf',
 't',
 'the',
 'tv',
 'x',
 'you',
 '~',
 '~!',
 '~!!',
 '~!!!',
 '~^^',
 '~~',
 '~~~',
 '~~~~',
 '♡

# 로지스틱 회귀

In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)

print("Training : ", lr.score(X_train, y_train))
print("Testing : ", lr.score(X_test, y_test))

Training :  0.8406266666666666
Testing :  0.82616


# MLP

In [25]:
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical

In [26]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [27]:
max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

In [28]:
model = Sequential()

model.add(Dense(64, input_shape=(max_words,)))
# model.add(Dense(64, input_dim=max_words))   # ===> 이것도 OK~!!!
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                192064    
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 196,354
Trainable params: 196,354
Non-trainable params: 0
__________________________________________________

In [29]:
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

In [30]:
# model.fit(X_train, y_train_cat,
# model.fit(X_train.A, y_train_cat,

model.fit(X_train.toarray(), y_train_cat,
        epochs=nb_epoch, 
        batch_size=batch_size) 

Epoch 1/5
1500/1500 [==============================] - 8s 2ms/step - loss: 0.3995 - accuracy: 0.8146
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3529 - accuracy: 0.8397
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3313 - accuracy: 0.8538
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3054 - accuracy: 0.8688
Epoch 5/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2746 - accuracy: 0.8833


### model.predict()
- 2차원 데이터(데이터수, 출력 수)

In [31]:
model.predict(X_test.toarray())

array([[2.7129399e-03, 9.9728703e-01],
       [6.3842058e-01, 3.6157939e-01],
       [8.2207903e-02, 9.1779214e-01],
       ...,
       [1.3438367e-03, 9.9865615e-01],
       [9.9999988e-01, 1.0746827e-07],
       [9.9377912e-01, 6.2208683e-03]], dtype=float32)

In [32]:
y_pred = np.argmax(model.predict(X_test.toarray()), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.83128

# RNN

### return_sequences=True
- 기본적으로 return_sequences가 False인 경우에는 SimpleRNN은 마지막 시점의 은닉 상태만 출력합니다. 이번에는 return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력
- (참고) https://wikidocs.net/106473

In [33]:
X_train.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
X_train

<150000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 1563112 stored elements in Compressed Sparse Row format>

In [35]:
# RNN 학습을 위한 데이터 재배열
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)
print(X_test_rnn.shape)

(150000, 1, 3000)
(50000, 1, 3000)


In [36]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()

model.add(SimpleRNN(128, input_shape=(X_train_rnn.shape[1], 
              X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 128)            400512    
_________________________________________________________________
activation_1 (Activation)    (None, 1, 128)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [37]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])   

In [38]:
model.fit(X_train_rnn, y_train_cat,
          batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
1500/1500 [==============================] - 20s 6ms/step - loss: 0.3952 - accuracy: 0.8156
Epoch 2/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3444 - accuracy: 0.8443
Epoch 3/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3050 - accuracy: 0.8668
Epoch 4/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2525 - accuracy: 0.8936
Epoch 5/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2024 - accuracy: 0.9151


In [39]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.8284